In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import random

# Load data
df = pd.read_csv('/content/IRIS.csv')
X = df.drop('species', axis=1).values
y = df['species'].values

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest parameters
n_trees = 10
max_features = int(np.sqrt(X.shape[1]))  # commonly used
tree_list = []

# Train each tree on a bootstrapped sample
for _ in range(n_trees):
    # Bootstrap sampling
    indices = np.random.choice(len(X_train), len(X_train), replace=True)
    X_sample = X_train[indices]
    y_sample = y_train[indices]

    # Select random subset of features
    feature_indices = np.random.choice(X.shape[1], max_features, replace=False)
    tree = DecisionTreeClassifier()
    tree.fit(X_sample[:, feature_indices], y_sample)
    tree_list.append((tree, feature_indices))

# Predict function with majority vote
def predict_forest(X):
    all_preds = []
    for x in X:
        votes = []
        for tree, f_idx in tree_list:
            pred = tree.predict([x[f_idx]])[0]
            votes.append(pred)
        # Majority vote
        majority_vote = Counter(votes).most_common(1)[0][0]
        all_preds.append(majority_vote)
    return np.array(all_preds)

# Make predictions
y_pred = predict_forest(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0
Classification Report:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30

